In [1]:
# this time just used spark not RDD at all.
df1 = spark.read.csv('/FileStore/tables/NCDB_2014_2016.csv', header=True, inferSchema=True)

1.
df1.show(5)

+----+---------+---------+-------------+-----------------+----------------+--------------------+-----------+-----------+---------+---------+--------------+------------------------+
Year| Month|DayOfWeek|CollisionHour|CollisionSeverity|VehiclesInvolved| WeatherCondition|RoadSurface|VehicleYear|PersonSex|PersonAge|PersonPosition|MedicalTreatmentRequired|
+----+---------+---------+-------------+-----------------+----------------+--------------------+-----------+-----------+---------+---------+--------------+------------------------+
2016| August| 6| 14| Non-fatal| 1| Clear and sunny|Dry, normal| 2002| M| 16| Driver| No Injury|
2016| February| 5| 20| Non-fatal| 1| Clear and sunny|Dry, normal| 2008| F| 16| Driver| No Injury|
2015| November| 3| 14| Non-fatal| 1|Visibility limita...|Dry, normal| 2008| F| 16| Driver| No Injury|
2015|September| 6| 12| Non-fatal| 1| Raining| Wet| 2009| M| 16| Driver| No Injury|
2016| November| 2| 4| Non-fatal| 1| Clear and sunny| Other| 2002| F| 17| Driver| No Injury|
+----+---------+---------+-------------+-----------------+----------------+--------------------+-----------+-----------+---------+---------+--------------+------------------------+
only showing top 5 rows

In [2]:
2. 
print("Count the number of collisions (i.e., rows) reported in the data: ",df1.count())

Count the number of collisions (i.e., rows) reported in the data: 740489

In [3]:
3. 
df1.select("CollisionSeverity").distinct().show()

+-----------------+
CollisionSeverity|
+-----------------+
 Fatal|
 Non-fatal|
+-----------------+

In [4]:
4. 
df1.groupby("Year").count().show()

+----+------+
Year| count|
+----+------+
2015|249224|
2014|239804|
2016|251461|
+----+------+

In [5]:
5. 
df2=df1.filter(df1['CollisionSeverity']=='Fatal')
from pyspark.sql.functions import mean, min, max
df2.select(mean("PersonAge").alias("Average Age")).show()

+-----------------+
 Average Age|
+-----------------+
41.35262771040059|
+-----------------+

In [6]:
6. # Not the same!
df3=df2.filter((df2['Month']=='December') & (df2['Year']==2014)).filter((df2["CollisionHour"]==2) | (df2["CollisionHour"]==3) )
df3.select(("PersonSex"),("PersonAge")).show()

+---------+---------+
PersonSex|PersonAge|
+---------+---------+
 M| 44|
 M| 46|
 M| 61|
 M| 51|
 M| 37|
 M| 30|
 M| 37|
+---------+---------+

In [7]:
7. 
df3=df2.groupby("PersonSex").count().show()

#df3=df3.withColumnRenamed("PersonSex", "count(PersonSex)").columns
#.show()

+---------+-----+
PersonSex|count|
+---------+-----+
 F| 3415|
 M| 7448|
 U| 21|
+---------+-----+

In [8]:
df1.show(10)

+----+---------+---------+-------------+-----------------+----------------+--------------------+-----------+-----------+---------+---------+--------------+------------------------+
Year| Month|DayOfWeek|CollisionHour|CollisionSeverity|VehiclesInvolved| WeatherCondition|RoadSurface|VehicleYear|PersonSex|PersonAge|PersonPosition|MedicalTreatmentRequired|
+----+---------+---------+-------------+-----------------+----------------+--------------------+-----------+-----------+---------+---------+--------------+------------------------+
2016| August| 6| 14| Non-fatal| 1| Clear and sunny|Dry, normal| 2002| M| 16| Driver| No Injury|
2016| February| 5| 20| Non-fatal| 1| Clear and sunny|Dry, normal| 2008| F| 16| Driver| No Injury|
2015| November| 3| 14| Non-fatal| 1|Visibility limita...|Dry, normal| 2008| F| 16| Driver| No Injury|
2015|September| 6| 12| Non-fatal| 1| Raining| Wet| 2009| M| 16| Driver| No Injury|
2016| November| 2| 4| Non-fatal| 1| Clear and sunny| Other| 2002| F| 17| Driver| No Injury|
2015| October| 3| 9| Fatal| 1| Clear and sunny|Dry, normal| 2005| F| 17| Driver| No Injury|
2014| October| 4| 15| Non-fatal| 1| Clear and sunny|Dry, normal| 2006| M| 17| Driver| No Injury|
2014| May| 5| 12| Non-fatal| 1| Clear and sunny|Dry, normal| 2007| F| 17| Driver| No Injury|
2015| May| 6| 16| Non-fatal| 1| Clear and sunny|Dry, normal| 2010| M| 17| Driver| No Injury|
2016| July| 7| 13| Non-fatal| 1| Overcast|Dry, normal| 2012| M| 17| Driver| No Injury|
+----+---------+---------+-------------+-----------------+----------------+--------------------+-----------+-----------+---------+---------+--------------+------------------------+
only showing top 10 rows

In [9]:
8.
df4 = df1.groupby("CollisionHour").count()
#df4.printSchema()
df4.orderBy(df4['count'].asc()).show(30)

+-------------+-----+
CollisionHour|count|
+-------------+-----+
 4| 4045|
 3| 5234|
 2| 6073|
 5| 6532|
 1| 6745|
 0| 9129|
 23|13141|
 6|15473|
 22|18013|
 21|23284|
 20|25556|
 7|30448|
 9|31818|
 19|33677|
 10|34076|
 11|40242|
 8|41081|
 13|47464|
 18|47662|
 12|48052|
 14|51935|
 15|64849|
 17|66487|
 16|69473|
+-------------+-----+

In [10]:
#df2.select(df2["PersonPosition"]).distinct().show()

+--------------------+
 PersonPosition|
+--------------------+
 Second row, left|
 Third row, left|
 Front row, right|
 Driver|
 Position unknown|
 Second row, center|
Outside passenger...|
 Third row, right|
 Second row, right|
 Third row, center|
Sitting on someon...|
 Front row, center|
+--------------------+

In [11]:
9.
df1.where(df1.PersonPosition.like("%left") | df1.PersonPosition.like("%right")).select(df1["Year"],df1["Month"],df1["PersonSex"],df1["PersonAge"]).orderBy(df1['Year'].asc(), df1['Month'].desc()).show(10)

+----+---------+---------+---------+
Year| Month|PersonSex|PersonAge|
+----+---------+---------+---------+
2014|September| F| 16|
2014|September| F| 21|
2014|September| M| 17|
2014|September| M| 16|
2014|September| F| 17|
2014|September| M| 8|
2014|September| U| 18|
2014|September| F| 18|
2014|September| M| 19|
2014|September| M| 13|
+----+---------+---------+---------+
only showing top 10 rows

In [12]:
10.
df1.groupby("Year","PersonSex").count().orderBy(df1['Year'].asc(), df1['PersonSex'].asc()).show()

+----+---------+------+
Year|PersonSex| count|
+----+---------+------+
2014| F|110786|
2014| M|128672|
2014| U| 346|
2015| F|114622|
2015| M|133931|
2015| U| 671|
2016| F|116075|
2016| M|134757|
2016| U| 629|
+----+---------+------+